In [ ]:
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import tensorflow as tf

import numpy as np
import seaborn as sns
import pandas as pd


In [ ]:
tf.__version__

In [ ]:
%load_ext tensorboard

In [ ]:
logdir= 'log'

In [ ]:
(train_images, train_labels), (test_images, test_labels) = datasets.mnist.load_data()

train_images = train_images.reshape((60000, 28,28,1))
test_images = test_images.reshape((10000,28,28,1))

train_images, test_images = train_images / 255.0, test_images / 255.0
classes = [0,1,2,3,4,5,6,7,8,9]

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3,3), activation='relu', input_shape=(28,28,1)))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64,(3,3), activation = 'relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64, (3,3), activation='relu'))

model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

In [ ]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x=train_images,
          y=train_labels,
          epochs=5,
          validation_data=(test_images, test_labels))

In [ ]:
# Faz predições (retorna um array de probabilidades para cada classe)
y_pred_probs = model.predict(test_images)

# Converte as probabilidades para classes (a que tiver maior valor)
y_pred = np.argmax(y_pred_probs, axis=1)

In [ ]:
classes=[0,1,2,3,4,5,6,7,8,9]

In [ ]:
con_mat = tf.math.confusion_matrix(labels=y_true, predictions=y_pred).numpy()
con_mat_norm = np.around(con_mat.astype('float') / con_mat.sum(axis=1)[:, np.newaxis], decimals=2)

con_mat_df = pd.DataFrame(con_mat_norm,
                          index = classes,
                          columns=classes)

figure = plt.figure(figsize=(8,8))
sns.heatmap(con_mat_df, annot=True, cmap=plt.cm.Blues)
plt.tight_layout()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

# Testes 

Sensibilidade VP / (VP + FN)

Especificidade VN / (FP + VN)

Acurácia (VP + VN) / N

Precisão VP / (VP + FP)

F-score  2x (PxS) / (P+S)

In [ ]:
# Sensibilidade

sensibilidade = np.diag(con_mat) / np.sum(con_mat, axis=1)

# Exibir
for idx, classe in enumerate(classes):
    print(f"Classe {classe} - Sensibilidade (Recall): {sensibilidade[idx]:.2f}")

In [ ]:
# Expecificidade

num_classes = con_mat.shape[0]
especificidade = []

for i in range(num_classes):
    TP = con_mat[i, i]
    FN = np.sum(con_mat[i, :]) - TP
    FP = np.sum(con_mat[:, i]) - TP
    TN = np.sum(con_mat) - (TP + FP + FN)

    esp = TN / (TN + FP) if (TN + FP) > 0 else 0
    especificidade.append(esp)


for idx, classe in enumerate(classes):
    print(f"Classe {classe} - Especificidade: {especificidade[idx]:.2f}")

In [ ]:
# Acurácia

acuracia_global = np.trace(con_mat) / np.sum(con_mat)
print(f"Acurácia global: {acuracia_global:.2f}")


num_classes = con_mat.shape[0]
acuracia_por_classe = []

for i in range(num_classes):
    TP = con_mat[i, i]
    FP = np.sum(con_mat[:, i]) - TP
    FN = np.sum(con_mat[i, :]) - TP
    TN = np.sum(con_mat) - (TP + FP + FN)

    acc = (TP + TN) / np.sum(con_mat)
    acuracia_por_classe.append(acc)

for idx, classe in enumerate(classes):
    print(f"Classe {classe} - Acurácia: {acuracia_por_classe[idx]:.2f}")



In [ ]:
# Precisão

num_classes = con_mat.shape[0]
precisao = []

for i in range(num_classes):
    TP = con_mat[i, i]
    FP = np.sum(con_mat[:, i]) - TP
    prec = TP / (TP + FP) if (TP + FP) > 0 else 0
    precisao.append(prec)

for idx, classe in enumerate(classes):
    print(f"Classe {classe} - Precisão: {precisao[idx]:.2f}")


In [ ]:
# F-score

num_classes = con_mat.shape[0]
f1_score = []

for i in range(num_classes):
    TP = con_mat[i, i]
    FN = np.sum(con_mat[i, :]) - TP
    FP = np.sum(con_mat[:, i]) - TP

    prec = TP / (TP + FP) if (TP + FP) > 0 else 0
    rec = TP / (TP + FN) if (TP + FN) > 0 else 0

    f1 = 2 * (prec * rec) / (prec + rec) if (prec + rec) > 0 else 0
    f1_score.append(f1)

for idx, classe in enumerate(classes):
    print(f"Classe {classe} - F1-Score: {f1_score[idx]:.2f}")
